## Preparazione dei dati
In questo notebook il dato excel di partenza viene modificato applicando alcune correzioni sul data type delle colonne e sul formato delle date per ottenere un dataset più ordinato ed organizzato

In [1]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import locale
import folium
import os

### Importazione del file Excel

In [2]:
data = os.path.join('../input/xls','wn.xlsx')
df = pd.read_excel(data)

### Correzioni colonne

In [3]:
# Modifica datatype delle colonne da float a int
df[
    ['AnnoFocolaioMinsal',
    'NumFocolaioMinsal',
    'CodSierotipo',
    'NumCapiAbbattuti',
    'NumCapiConSintomi',
    'NumCapiDistrutti',
    'NumCapiGuariti',
    'NumCapiMalati',
    'NumCapiMorti',
    'NumCapiPresenti',
    'TotaleMorti']
] = df[
    [
    'AnnoFocolaioMinsal',
    'NumFocolaioMinsal',
    'CodSierotipo',
    'NumCapiAbbattuti',
    'NumCapiConSintomi',
    'NumCapiDistrutti',
    'NumCapiGuariti',
    'NumCapiMalati',
    'NumCapiMorti',
    'NumCapiPresenti',
    'TotaleMorti']
].fillna(0).astype(int)

# Modifica datatype di CodIstat in stringa
df['CodIstat'] = df['CodIstat'].astype('str')

# Trasformazione in data da stringa DD-MMM-YY a YYYY-MM-DD
locale.setlocale(locale.LC_ALL, 'it_IT')
df['DataSospetto'] = pd.to_datetime(df['DataSospetto'], format='%d-%b-%y')
df['DataInizioSintomi'] = pd.to_datetime(df['DataInizioSintomi'], format='%d-%b-%y')
df['DataConferma'] = pd.to_datetime(df['DataConferma'], format='%d-%b-%y')
df['DataEstinzione'] = pd.to_datetime(df['DataEstinzione'], format='%d-%b-%y')

# Crea campo AnnoSospetto
df['AnnoSospetto'] = pd.DatetimeIndex(df['DataSospetto']).year

# Rimozione colonne inutili
df.drop(['STATO_DEL_RECORD','Malattia'], axis = 1, inplace=True)

### Dataset di partenza corretto
Mantiene i soli focolai confermati, ordina le righe per data decrescente e riordina e colonne

In [4]:
df_clean = df.query('DataConferma != "NaT"')[
    ['AnnoSospetto', 'DataSospetto','DataConferma',
     'IdFocolaio','CodAzienda','CoordX', 'CoordY', 'CodIstat',
     'Regione', 'Prov', 'Comune',
     'Specie','Categorie', 'CodSierotipo', 'Sierotipo']].sort_values(by=['DataConferma'], ascending=False)

In [5]:
df_clean.head(3)

,AnnoSospetto,DataSospetto,DataConferma,IdFocolaio,CodAzienda,CoordX,CoordY,CodIstat,Regione,Prov,Comune,Specie,Categorie,CodSierotipo,Sierotipo
1945,2020,2020-10-29,2020-11-27,48394,014MI007,9.449249,45.588372,15014,LOMBARDIA,MI,BASIANO,GAZZA,UCCELLI TARGET,2,Lineage 2
0,2020,2020-10-25,2020-11-26,48429,AL22122D,8.463033,45.357862,2017,PIEMONTE,VC,BORGO VERCELLI,CORNACCHIA,UCCELLI TARGET,2,Lineage 2
1943,2020,2020-09-05,2020-11-26,48039,AL21081D,9.107900,45.275000,18173,LOMBARDIA,PV,VELLEZZO BELLINI,SPARVIERO,UCCELLI SELVATICI,2,Lineage 2


In [9]:
xls_writer = pd.ExcelWriter("../input/xls/wn_clean.xlsx", datetime_format='dd/mm/yyyy')
df_clean.to_excel(xls_writer, "Sheet1", index=False)
xls_writer.close()

### Creazione dataset spaziale
Crea un geodataframe utilizzando le coordinate contenute nel dataset e lo esporta sotto forma di **GeoJSON** e **GeoPKG**

In [13]:
gdf = gpd.GeoDataFrame(df_clean, geometry=gpd.points_from_xy(df_clean.CoordX, df_clean.CoordY), crs={"init":"EPSG:4326"})
gdf.drop(['CoordX','CoordY'], axis = 1, inplace=True)

/Users/alessio/opt/anaconda3/envs/report_wn/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [14]:
gdf.head(3)

,AnnoSospetto,DataSospetto,DataConferma,IdFocolaio,CodAzienda,CodIstat,Regione,Prov,Comune,Specie,Categorie,CodSierotipo,Sierotipo,geometry
1945,2020,2020-10-29,2020-11-27,48394,014MI007,15014,LOMBARDIA,MI,BASIANO,GAZZA,UCCELLI TARGET,2,Lineage 2,POINT (9.44925 45.58837)
0,2020,2020-10-25,2020-11-26,48429,AL22122D,2017,PIEMONTE,VC,BORGO VERCELLI,CORNACCHIA,UCCELLI TARGET,2,Lineage 2,POINT (8.46303 45.35786)
1943,2020,2020-09-05,2020-11-26,48039,AL21081D,18173,LOMBARDIA,PV,VELLEZZO BELLINI,SPARVIERO,UCCELLI SELVATICI,2,Lineage 2,POINT (9.10790 45.27500)


In [12]:
# gdf.to_file("../output/focolai_wn.geojson", driver="GeoJSON")
# gdf.to_file("../output/focolai_wn.gpkg", layer="focolai", driver="GPKG")

DriverError: Failed to create GeoJSON datasource: ../output/focolai_wn.geojson: ../output/focolai_wn.geojson: No such file or directory

### Verifica Punti

In [12]:
# Trasformazione dei campi datetime in stringhe per permettere la serializzazione come oggetti GeoJSON
gdf[['DataSospetto','DataConferma']] = gdf[['DataSospetto','DataConferma']].astype(str)

In [13]:
map = folium.Map([42.783333, 12.866667], zoom_start=5, tiles='openstreetmap')

focolai = folium.features.GeoJson(gdf.to_json(), name='Focolai West Nile', show=True)
map.add_child(focolai)

map.add_child(folium.LayerControl())

map